# Inference of MicroRNA-Messenger RNA Interactions in TCGA-BRCA
TCGA: The Cancer Genome Atlas  
BRCA: Breast Invasive Carcinoma

# Importing Libraries and Configurations

In [1]:
import os
import sys

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from scipy.stats import false_discovery_control, spearmanr

# Add project root to Python's path
sys.path.append(os.path.abspath(os.path.join('..', '..')))

from config import (
    AGGREGATED_READS_FILES,
    BRCA_PROCESSED_FILES_DIRS,
    BRCA_PROCESSED_FILES_PATHS,
    INTERACTION_INFERENCE_PARAMETERS,
    MIRWALK_MIR_MAPPING_FILE_PATH,
    MIRWALK_PROCESSED_DATA_DIR,
)

# Function

In [2]:
def prepare_data_for_spearman(processed_dir_path):
    """
    Prepare aggregated expression data of a group for Spearman correlation analysis.

    Parameters
    ----------
    processed_dir_path : str
        Path to the directory containing processed expression data files.

    Returns
    -------
    dict of pd.DataFrame
        A dictionary with keys:
        - 'mir' : DataFrame with aggregated microRNA normalized read counts.
        - 'rna' : DataFrame with aggregated messenger RNA normalized read counts.
        Both DataFrames have case IDs as indices and molecule identifiers as columns.
    """
    # Create a DataFrame for the files metadata
    df_files_metadata = pd.read_csv(BRCA_PROCESSED_FILES_PATHS['files'])

    # Initialize a dictionary for Spearman-ready DataFrames
    aggregated_reads = dict()

    # Prepare the aggregated reads files
    for experimental_strategy in ['mir', 'rna']:
        # Define the experimental strategy related parameters
        drop_column = ['is_expressed']
        if experimental_strategy == 'mir':
            index_column = 'accession_id'
            file_name = AGGREGATED_READS_FILES['mir-normalized']
        else:
            index_column = 'gene_name'
            drop_column = drop_column + ['gene_id']
            file_name = AGGREGATED_READS_FILES['rna-normalized']
        
        # Get the expressed molecules and set the primary key as index
        file_path = os.path.join(processed_dir_path, file_name)
        df_agg_reads = pd.read_csv(file_path) \
            .query('is_expressed == 1') \
            .drop(columns=drop_column) \
            .set_index(keys=index_column)
        
        # Map the file ID to the associated case ID
        file_ids = list(df_agg_reads.columns)
        df_files_mapping = pd.DataFrame(file_ids, columns=['file_id'])
        df_files_mapping = df_files_mapping \
            .merge(
                right=df_files_metadata,
                left_on='file_id',
                right_on='file_id',
                how='left',
            ) \
            [['file_id', 'case_id']]
        
        # Transpose the DataFrame and set the case ID as index
        df_agg_reads = df_agg_reads \
            .transpose() \
            .reset_index() \
            .rename(columns={'index': 'file_id'}) \
            .merge(
                right=df_files_mapping,
                left_on='file_id',
                right_on='file_id',
                how='inner',
            ) \
            .drop(columns=['file_id']) \
            .set_index(keys='case_id') \
            .sort_index(ascending=True)
        
        # Store the prepared DataFrame in the dictionary
        aggregated_reads[experimental_strategy] = df_agg_reads.copy()
    
    return aggregated_reads

In [3]:
def compute_single_pair_spearman(ser_mir_expression, ser_rna_expression):
    """
    Calculate the Spearman correlation coefficient between the expression profile 
    of a single microRNA and messenger RNA pair.
    
    Parameters
    ----------
    ser_mir_expression : pd.Series
        Expression values of a single microRNA across samples.
    ser_rna_expression : pd.Series
        Expression values of a single messenger RNA across the same samples.

    Returns
    -------
    dict
        A dictionary containing:
        - 'accession_id': microRNA accession ID.
        - 'gene_name': gene name of the messenger RNA.
        - 'correlation': Spearman correlation coefficient.
        - 'pvalue': p-value for the hypothesis test that correlation < 0.
    """
    # Calculate the Spearman correlation coefficient for the pair
    axis = INTERACTION_INFERENCE_PARAMETERS['axis']
    alternative = INTERACTION_INFERENCE_PARAMETERS['alternative']
    correlation, pvalue = spearmanr(
        a=ser_mir_expression, # Samples of the expressed microRNA
        b=ser_rna_expression, # Samples of the expressed messenger RNAs
        axis=axis, # Each row is a observation, while the columns are variables
        alternative=alternative, # The correlation is negative in miRNA-mRNA inferred_interactions
    )
    
    # Create a dictionary to represent the results
    results = {
        'accession_id': ser_mir_expression.name,
        'gene_name': ser_rna_expression.name,
        'correlation': correlation,
        'pvalue': pvalue,
    }
    
    return results

In [4]:
def infer_mir_rna_interactions(df_mirwalk_interactions, group):
    """
    Calculate Spearman correlation coefficients between all pairs of expressed microRNAs and 
    messenger RNAs for a given group using parallel processing.

    Parameter
    ---------
    group : str
        Name of the group used to locate interim and processed data directories.

    Returns
    -------
    pd.DataFrame
        A DataFrame containing the Spearman correlation results with one row per microRNA-
        messenger RNA pair and their corresponding correlation values.
    """
    # Define the group interim and processed directories path
    dir_base_name = (group.lower()).replace(' ', '-')
    processed_dir_path = BRCA_PROCESSED_FILES_DIRS[dir_base_name]
    
    # Prepare the expression data of the expressed molecules
    aggregated_reads = prepare_data_for_spearman(processed_dir_path)
    df_mir_expression = aggregated_reads['mir']
    df_rna_expression = aggregated_reads['rna']
    
    # Define all potential inferred_interactions between expressed microRNAs and messenger RNAs
    df_expressed_mirs = pd.DataFrame(
        data=list(df_mir_expression.columns), columns=['accession_id']
    )
    df_expressed_rnas = pd.DataFrame(
        data=list(df_rna_expression.columns), columns=['gene_name']
    )
    df_potential_inferred_interactions = df_expressed_mirs \
        .merge(right=df_expressed_rnas, how='cross')
    
    # Select inferred_interactions with miRWalk retrieved evidence
    primary_key = ['accession_id', 'gene_name']
    df_inferred_interactions = df_potential_inferred_interactions \
        .merge(
            right=df_mirwalk_interactions,
            left_on=primary_key,
            right_on=primary_key,
            how='inner',
        )
    group_inferred_interactions = zip(
        df_inferred_interactions['accession_id'], df_inferred_interactions['gene_name'],
    )
    
    # Compute in parallel the Spearman correlation coefficient for each pair
    results = Parallel(n_jobs=-1, prefer='processes')(
        delayed(compute_single_pair_spearman)
        (df_mir_expression[mir], df_rna_expression[rna])
        for mir, rna in group_inferred_interactions
    )
    
    # Create a DataFrame for the computing results
    df_results = pd.DataFrame(results)
    
    # Adjust the p-values to control the false discovery fate (FDR)
    axis = INTERACTION_INFERENCE_PARAMETERS['axis']
    method = INTERACTION_INFERENCE_PARAMETERS['fdr-method']
    qvalues = false_discovery_control(
        ps=df_results['pvalue'], # The p-values to adjust
        axis=axis, # The axis along which to perform the adjustment
        method=method, # FDR control procedure
    )
    df_results['qvalue'] = qvalues
    
    # Add miRWalk data to results
    df_results = df_mirwalk_interactions \
        .merge(
            right=df_results,
            left_on=primary_key,
            right_on=primary_key,
            how='inner',
        )
        
    # Flag the inferred interactions of interest, i.e. those with correlation and 
    # q-value values below the thresholds
    min_correlation = INTERACTION_INFERENCE_PARAMETERS['min_correlation']
    min_qvalue = INTERACTION_INFERENCE_PARAMETERS['min_qvalue']
    df_results['is_interaction_of_interest'] = np.where(
        ((df_results['correlation'] < min_correlation)
         & (df_results['qvalue'] < min_qvalue)), 1, 0
    )
    
    # Store the DataFrame of inferred interactions
    file_name = INTERACTION_INFERENCE_PARAMETERS['file']
    df_results.to_csv(os.path.join(processed_dir_path, file_name), index=False)
    
    return df_results

# miRWalk Interactions of Interest

In [5]:
# Initialize a DataFrame for the inferred interactions of interest from miRWalk
mirwalk_columns = ['mirna_name', 'gene_name', 'mirtarbase']
df_mirwalk_interactions = pd.DataFrame(columns=mirwalk_columns)

# List the interaction files downloaded from miRWalk
files = [f for f in os.listdir(MIRWALK_PROCESSED_DATA_DIR) if f.startswith('MIMAT')]

# Iterate over each inferred_interactions file from miRWalk
for file in files:
    # Create a DataFrame for the inferred interactions of interest of the microRNA
    file_path = os.path.join(MIRWALK_PROCESSED_DATA_DIR, file)    
    df_mir_inferred_interactions = pd.read_csv(file_path, low_memory=False) \
        .query('is_interaction_of_interest == 1') \
        [mirwalk_columns] \
        .drop_duplicates()
    
    # Concatenate the inferred interactions of interest to the others
    df_mirwalk_interactions = pd.concat(
        [df_mirwalk_interactions, df_mir_inferred_interactions], ignore_index=True
    )

# Add the microRNA accession IDs to the DataFrame
df_mir_mapping = pd.read_csv(MIRWALK_MIR_MAPPING_FILE_PATH)
df_mirwalk_interactions = df_mir_mapping \
    .merge(
        right=df_mirwalk_interactions,
        left_on='mirna_name',
        right_on='mirna_name',
        how='inner',
    )

In [6]:
# Print the DataFrame of inferred_interactions of interest from miRWalk
df_mirwalk_interactions

,accession_id,mirna_name,gene_name,mirtarbase
0,MIMAT0000062,hsa-let-7a-5p,PKIA,NaN
1,MIMAT0000062,hsa-let-7a-5p,GJC1,NaN
2,MIMAT0000062,hsa-let-7a-5p,PBX3,NaN
3,MIMAT0000062,hsa-let-7a-5p,ITSN1,NaN
4,MIMAT0000062,hsa-let-7a-5p,SRGAP1,NaN
...,...,...,...,...
41848,MIMAT0026483,hsa-miR-370-5p,STUM,NaN
41849,MIMAT0026483,hsa-miR-370-5p,CRISPLD2,NaN
41850,MIMAT0026483,hsa-miR-370-5p,ADAM19,NaN
41851,MIMAT0026557,hsa-miR-412-5p,FAM53C,NaN


# Interaction Inference

## Basal-like

In [7]:
# Infer the interactions for Basal-like tumor tissue
inferred_interactions = infer_mir_rna_interactions(df_mirwalk_interactions, 'Basal-like')

In [8]:
# Print the inferred interactions associated with this group
inferred_interactions

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue,is_interaction_of_interest
0,MIMAT0000062,hsa-let-7a-5p,PKIA,NaN,-0.078607,0.234621,0.745015,0
1,MIMAT0000062,hsa-let-7a-5p,GJC1,NaN,0.018153,0.566268,0.978192,0
2,MIMAT0000062,hsa-let-7a-5p,PBX3,NaN,0.143526,0.907620,1.000000,0
3,MIMAT0000062,hsa-let-7a-5p,ITSN1,NaN,-0.033079,0.380501,0.873011,0
4,MIMAT0000062,hsa-let-7a-5p,SRGAP1,NaN,0.047532,0.669012,1.000000,0
...,...,...,...,...,...,...,...,...
41164,MIMAT0026483,hsa-miR-370-5p,STUM,NaN,0.092500,0.802930,1.000000,0
41165,MIMAT0026483,hsa-miR-370-5p,CRISPLD2,NaN,0.430324,0.999984,1.000000,0
41166,MIMAT0026483,hsa-miR-370-5p,ADAM19,NaN,0.221807,0.980527,1.000000,0
41167,MIMAT0026557,hsa-miR-412-5p,FAM53C,NaN,-0.016884,0.438326,0.911847,0


In [9]:
# Print the inferred interactions of interest associated with this group
inferred_interactions \
    .query('is_interaction_of_interest == 1') \
    .drop(columns=['is_interaction_of_interest']) \
    .reset_index(drop=True)

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue
0,MIMAT0000064,hsa-let-7c-5p,IGF1R,MIRT053087,-0.388423,0.000101,0.028263
1,MIMAT0000064,hsa-let-7c-5p,EIF2S2,NaN,-0.416928,0.000029,0.017575
2,MIMAT0000065,hsa-let-7d-5p,ITSN1,NaN,-0.378399,0.000151,0.033543
3,MIMAT0000066,hsa-let-7e-5p,CD86,NaN,-0.367774,0.000229,0.039746
4,MIMAT0000066,hsa-let-7e-5p,CD200R1,NaN,-0.372494,0.000191,0.037333
...,...,...,...,...,...,...,...
291,MIMAT0002820,hsa-miR-497-5p,HMGA1,NaN,-0.392998,0.000083,0.026621
292,MIMAT0002820,hsa-miR-497-5p,EIF3M,NaN,-0.401217,0.000059,0.023656
293,MIMAT0003301,hsa-miR-33b-5p,TARDBP,NaN,-0.400115,0.000062,0.023656
294,MIMAT0004693,hsa-miR-330-5p,NGFR,NaN,-0.375501,0.000169,0.035921


## HER2-enriched

In [10]:
# Infer the interactions for HER2-enriched tumor tissue
inferred_interactions = infer_mir_rna_interactions(df_mirwalk_interactions, 'HER2-enriched')

In [11]:
# Print the inferred interactions associated with this group
inferred_interactions

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue,is_interaction_of_interest
0,MIMAT0000062,hsa-let-7a-5p,PKIA,NaN,-0.013944,0.459379,0.871133,0
1,MIMAT0000062,hsa-let-7a-5p,GJC1,NaN,0.248189,0.967434,1.000000,0
2,MIMAT0000062,hsa-let-7a-5p,PBX3,NaN,0.003896,0.511368,0.894937,0
3,MIMAT0000062,hsa-let-7a-5p,ITSN1,NaN,0.102802,0.774561,0.982053,0
4,MIMAT0000062,hsa-let-7a-5p,SRGAP1,NaN,-0.067943,0.309403,0.789843,0
...,...,...,...,...,...,...,...,...
41164,MIMAT0026483,hsa-miR-370-5p,STUM,NaN,0.378153,0.997971,1.000000,0
41165,MIMAT0026483,hsa-miR-370-5p,CRISPLD2,NaN,0.378153,0.997971,1.000000,0
41166,MIMAT0026483,hsa-miR-370-5p,ADAM19,NaN,0.262629,0.974737,1.000000,0
41167,MIMAT0026557,hsa-miR-412-5p,FAM53C,NaN,0.236705,0.960494,1.000000,0


In [12]:
# Print the inferred interactions of interest associated with this group
inferred_interactions \
    .query('is_interaction_of_interest == 1') \
    .drop(columns=['is_interaction_of_interest']) \
    .reset_index(drop=True)

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue
0,MIMAT0000065,hsa-let-7d-5p,HIF1AN,NaN,-0.539303,8.986564e-06,0.041561
1,MIMAT0000065,hsa-let-7d-5p,MEF2D,MIRT735654,-0.562406,3.225376e-06,0.032134
2,MIMAT0000065,hsa-let-7d-5p,SMARCC1,NaN,-0.546275,6.649086e-06,0.041507
3,MIMAT0000065,hsa-let-7d-5p,PRKAR2A,NaN,-0.536637,1.006593e-05,0.041561
4,MIMAT0000072,hsa-miR-18a-5p,ZBTB20,NaN,-0.558715,3.818749e-06,0.032134
5,MIMAT0000244,hsa-miR-30c-5p,SNAI1,MIRT006762,-0.529050,1.382919e-05,0.043795
6,MIMAT0000259,hsa-miR-182-5p,GPHN,NaN,-0.531852,1.230922e-05,0.043795
7,MIMAT0000432,hsa-miR-141-3p,IGF2,NaN,-0.587491,9.681125e-07,0.032134
8,MIMAT0000646,hsa-miR-155-5p,VAV3,NaN,-0.536569,1.009512e-05,0.041561
9,MIMAT0000680,hsa-miR-106b-5p,PLXDC2,NaN,-0.558237,3.902691e-06,0.032134


## Luminal A

In [13]:
# Infer the interactions for Luminal A tumor tissue
inferred_interactions = infer_mir_rna_interactions(df_mirwalk_interactions, 'Luminal A')

In [14]:
# Print the inferred interactions associated with this group
inferred_interactions

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue,is_interaction_of_interest
0,MIMAT0000062,hsa-let-7a-5p,PKIA,NaN,-0.010777,0.436428,0.841128,0
1,MIMAT0000062,hsa-let-7a-5p,GJC1,NaN,-0.043499,0.259064,0.648037,0
2,MIMAT0000062,hsa-let-7a-5p,PBX3,NaN,0.059124,0.810222,1.000000,0
3,MIMAT0000062,hsa-let-7a-5p,ITSN1,NaN,0.061249,0.818681,1.000000,0
4,MIMAT0000062,hsa-let-7a-5p,SRGAP1,NaN,-0.035360,0.299708,0.697952,0
...,...,...,...,...,...,...,...,...
41164,MIMAT0026483,hsa-miR-370-5p,STUM,NaN,0.143357,0.983815,1.000000,0
41165,MIMAT0026483,hsa-miR-370-5p,CRISPLD2,NaN,0.456634,1.000000,1.000000,0
41166,MIMAT0026483,hsa-miR-370-5p,ADAM19,NaN,0.252987,0.999933,1.000000,0
41167,MIMAT0026557,hsa-miR-412-5p,FAM53C,NaN,0.017501,0.602526,0.970287,0


In [15]:
# Print the inferred interactions of interest associated with this group
inferred_interactions \
    .query('is_interaction_of_interest == 1') \
    .drop(columns=['is_interaction_of_interest']) \
    .reset_index(drop=True)

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue
0,MIMAT0000064,hsa-let-7c-5p,PARD6B,NaN,-0.305317,1.699354e-06,0.000311
1,MIMAT0000069,hsa-miR-16-5p,TLE4,MIRT031643,-0.315052,7.889337e-07,0.000169
2,MIMAT0000069,hsa-miR-16-5p,PLSCR4,MIRT031585,-0.338303,1.127362e-07,0.000046
3,MIMAT0000069,hsa-miR-16-5p,ETV1,NaN,-0.325088,3.474575e-07,0.000097
4,MIMAT0000069,hsa-miR-16-5p,ACVR2A,MIRT000536,-0.312958,9.326604e-07,0.000193
...,...,...,...,...,...,...,...
254,MIMAT0003301,hsa-miR-33b-5p,CROT,NaN,-0.302597,2.095452e-06,0.000356
255,MIMAT0003301,hsa-miR-33b-5p,SLC16A7,NaN,-0.323011,4.127512e-07,0.000109
256,MIMAT0003393,hsa-miR-425-5p,PPP2CB,MIRT016657,-0.310217,1.158869e-06,0.000233
257,MIMAT0004945,hsa-miR-744-5p,HPSE2,NaN,-0.378697,2.572259e-09,0.000003


## Luminal B

In [16]:
# Infer the interactions for Luminal B tumor tissue
inferred_interactions = infer_mir_rna_interactions(df_mirwalk_interactions, 'Luminal B')

In [17]:
# Print the inferred interactions associated with this group
inferred_interactions

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue,is_interaction_of_interest
0,MIMAT0000062,hsa-let-7a-5p,PKIA,NaN,-0.162456,0.038132,0.262916,0
1,MIMAT0000062,hsa-let-7a-5p,GJC1,NaN,0.096166,0.851953,1.000000,0
2,MIMAT0000062,hsa-let-7a-5p,PBX3,NaN,0.064838,0.759150,0.995618,0
3,MIMAT0000062,hsa-let-7a-5p,ITSN1,NaN,0.149260,0.948140,1.000000,0
4,MIMAT0000062,hsa-let-7a-5p,SRGAP1,NaN,0.173825,0.971198,1.000000,0
...,...,...,...,...,...,...,...,...
41164,MIMAT0026483,hsa-miR-370-5p,STUM,NaN,0.076256,0.796110,1.000000,0
41165,MIMAT0026483,hsa-miR-370-5p,CRISPLD2,NaN,0.317266,0.999793,1.000000,0
41166,MIMAT0026483,hsa-miR-370-5p,ADAM19,NaN,0.141708,0.938692,1.000000,0
41167,MIMAT0026557,hsa-miR-412-5p,FAM53C,NaN,-0.070633,0.221659,0.598784,0


In [18]:
# Print the inferred interactions of interest associated with this group
inferred_interactions \
    .query('is_interaction_of_interest == 1') \
    .drop(columns=['is_interaction_of_interest']) \
    .reset_index(drop=True)

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue
0,MIMAT0000063,hsa-let-7b-5p,SENP5,NaN,-0.305959,0.000339,0.027519
1,MIMAT0000063,hsa-let-7b-5p,PEG10,MIRT561775,-0.351302,0.000042,0.010108
2,MIMAT0000065,hsa-let-7d-5p,PDE12,MIRT120920,-0.301715,0.000406,0.029633
3,MIMAT0000065,hsa-let-7d-5p,CALU,MIRT123316,-0.300611,0.000425,0.030313
4,MIMAT0000065,hsa-let-7d-5p,CSRNP3,NaN,-0.316057,0.000219,0.022328
...,...,...,...,...,...,...,...
579,MIMAT0016847,hsa-miR-378c,TOLLIP,NaN,-0.317707,0.000203,0.021596
580,MIMAT0016847,hsa-miR-378c,MEF2D,NaN,-0.330541,0.000113,0.016482
581,MIMAT0016847,hsa-miR-378c,ABI2,NaN,-0.312810,0.000252,0.024002
582,MIMAT0018443,hsa-miR-374c-5p,ATXN1,NaN,-0.304957,0.000354,0.027953


## Paired Normal

In [19]:
# Infer the interactions for Paired Normal tissue
inferred_interactions = infer_mir_rna_interactions(df_mirwalk_interactions, 'Paired Normal')

In [20]:
# Print the inferred interactions associated with this group
inferred_interactions

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue,is_interaction_of_interest
0,MIMAT0000062,hsa-let-7a-5p,PKIA,NaN,0.237936,0.961289,1.000000,0
1,MIMAT0000062,hsa-let-7a-5p,GJC1,NaN,-0.015379,0.455214,0.898533,0
2,MIMAT0000062,hsa-let-7a-5p,PBX3,NaN,0.007314,0.521332,0.969059,0
3,MIMAT0000062,hsa-let-7a-5p,ITSN1,NaN,-0.077102,0.286103,0.674410,0
4,MIMAT0000062,hsa-let-7a-5p,SRGAP1,NaN,0.162748,0.884626,1.000000,0
...,...,...,...,...,...,...,...,...
41164,MIMAT0026483,hsa-miR-370-5p,STUM,NaN,0.198770,0.929030,1.000000,0
41165,MIMAT0026483,hsa-miR-370-5p,CRISPLD2,NaN,0.176897,0.903923,1.000000,0
41166,MIMAT0026483,hsa-miR-370-5p,ADAM19,NaN,0.176555,0.903485,1.000000,0
41167,MIMAT0026557,hsa-miR-412-5p,FAM53C,NaN,0.143814,0.854845,1.000000,0


In [21]:
# Print the inferred interactions of interest associated with this group
inferred_interactions \
    .query('is_interaction_of_interest == 1') \
    .drop(columns=['is_interaction_of_interest']) \
    .reset_index(drop=True)

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue
0,MIMAT0000062,hsa-let-7a-5p,GIPC1,NaN,-0.427751,5.038145e-04,0.008435
1,MIMAT0000062,hsa-let-7a-5p,MMP11,NaN,-0.339986,5.178013e-03,0.044181
2,MIMAT0000063,hsa-let-7b-5p,GJC1,NaN,-0.598291,5.586415e-07,0.000054
3,MIMAT0000063,hsa-let-7b-5p,MEF2C,MIRT052262,-0.340807,5.080635e-03,0.043603
4,MIMAT0000063,hsa-let-7b-5p,SOX6,NaN,-0.426589,5.218179e-04,0.008635
...,...,...,...,...,...,...,...
5067,MIMAT0026483,hsa-miR-370-5p,KCNB1,NaN,-0.355776,3.561907e-03,0.033726
5068,MIMAT0026483,hsa-miR-370-5p,ANTXR2,NaN,-0.337184,5.522999e-03,0.046262
5069,MIMAT0026483,hsa-miR-370-5p,PIP4P2,NaN,-0.334586,5.860281e-03,0.048118
5070,MIMAT0026483,hsa-miR-370-5p,ATP1A2,NaN,-0.339986,5.178013e-03,0.044181
